In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/cimat-ris/OF-PathPred.git clonedrep

# Change directory into cloned repository
%cd clonedrep/tests

C:\Users\jose0\Desktop\OF-PathPred\tests\clonedrep\tests


Cloning into 'clonedrep'...

Updating files:  21% (31/141)

Updating files:  22% (32/141)

Updating files:  23% (33/141)

Updating files:  24% (34/141)

Updating files:  25% (36/141)

Updating files:  26% (37/141)

Updating files:  27% (39/141)

Updating files:  28% (40/141)

Updating files:  29% (41/141)

Updating files:  30% (43/141)

Updating files:  31% (44/141)

Updating files:  32% (46/141)

Updating files:  33% (47/141)

Updating files:  34% (48/141)

Updating files:  35% (50/141)

Updating files:  36% (51/141)

Updating files:  37% (53/141)

Updating files:  38% (54/141)

Updating files:  39% (55/141)

Updating files:  40% (57/141)

Updating files:  41% (58/141)

Updating files:  42% (60/141)

Updating files:  43% (61/141)

Updating files:  44% (63/141)

Updating files:  45% (64/141)

Updating files:  46% (65/141)

Updating files:  47% (67/141)

Updating files:  48% (68/141)

Updating files:  49% (70/141)

Updating files:  50% (71/141)

Updating files:  51% (72/141)

Updating f

In [ ]:
# Imports
import sys,os
''' TF_CPP_MIN_LOG_LEVEL
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed
'''
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
sys.path.append('../')
import math,numpy as np
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print('[INF] Tensorflow version: ',tf.__version__)
tf.test.gpu_device_name()
# Important imports
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from path_prediction.datasets_utils import setup_loo_experiment
from path_prediction.model import TrajectoryEncoderDecoder, Model_Parameters
from path_prediction.plot_utils import plot_training_data,plot_training_results
import path_prediction.batches_data
from path_prediction.testing_utils import evaluation_minadefde,evaluation_qualitative,evaluation_attention,plot_comparisons_minadefde, get_testing_batch
from path_prediction.training_utils import training_loop
from path_prediction.training_utils import Experiment_Parameters

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices)>0:
    print('[INF] Using GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("[INF] Using CPU")

In [ ]:
# Load the default parameters
experiment_parameters = Experiment_Parameters(add_social=False,add_kp=False,obstacles=False)

dataset_dir   = "../datasets/"
dataset_names = ['eth-hotel','eth-univ','ucy-zara01','ucy-zara02','ucy-univ']

In [ ]:
# Load the dataset and perform the split
idTest = 2
training_data,validation_data,test_data,test_homography = setup_loo_experiment('ETH_UCY',dataset_dir,dataset_names,idTest,experiment_parameters,use_pickled_data=False,pickle_dir="../pickle")


In [ ]:
#############################################################
# Model parameters
model_parameters = Model_Parameters(add_attention=True,add_kp=experiment_parameters.add_kp,add_social=experiment_parameters.add_social,output_representation=experiment_parameters.output_representation)
model_parameters.num_epochs     = 35
model_parameters.output_var_dirs= 4
model_parameters.is_mc_dropout  = True
model_parameters.initial_lr     = 0.03

# Running on CPU
if len(physical_devices)==0:
    model_parameters.batch_size     = 64
    model_parameters.output_var_dirs= 1
    model_parameters.stack_rnn_size = 1


In [ ]:
# Get the necessary data
train_data = tf.data.Dataset.from_tensor_slices(training_data)
val_data   = tf.data.Dataset.from_tensor_slices(validation_data)
test_data  = tf.data.Dataset.from_tensor_slices(test_data)


In [ ]:
# Form batches
batched_train_data = train_data.batch(model_parameters.batch_size)
batched_val_data   = val_data.batch(model_parameters.batch_size)
batched_test_data  = test_data.batch(model_parameters.batch_size)


In [ ]:
# Model
tj_enc_dec = TrajectoryEncoderDecoder(model_parameters)


In [ ]:
# Checkpoints
checkpoint_dir   = '../training_checkpoints'
checkpoint_prefix= os.path.join(checkpoint_dir, "ckpt")
checkpoint       = tf.train.Checkpoint(optimizer=tj_enc_dec.optimizer,
                                        encoder=tj_enc_dec.enc,
                                        decoder=tj_enc_dec.dec,
                                        enctodec=tj_enc_dec.enctodec,
                                        obs_classif=tj_enc_dec.obs_classif)


In [ ]:
# Training
perform_training = True
plot_training    = True
if perform_training==True:
    print("[INF] Training the model")
    train_loss_results,val_loss_results,val_metrics_results,__ = training_loop(tj_enc_dec,batched_train_data,batched_val_data,model_parameters,checkpoint,checkpoint_prefix)
    if plot_training==True:
        plot_training_results(train_loss_results,val_loss_results,val_metrics_results)


In [ ]:
# Testing
# Restoring the latest checkpoint in checkpoint_dir
print("[INF] Restoring last model")
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


In [ ]:
# Quantitative testing: ADE/FDE
print("[INF] Quantitative testing")
results = evaluation_minadefde(tj_enc_dec,batched_test_data,model_parameters)
plot_comparisons_minadefde(results,dataset_names[idTest])
print(results)


In [ ]:
# Qualitative testing
qualitative = True
if qualitative==True:
    print("[INF] Qualitative testing")
    for i in range(10):
        batch, test_bckgd = get_testing_batch(test_data,dataset_dir+dataset_names[idTest])
        evaluation_qualitative(tj_enc_dec,batch,model_parameters,background=test_bckgd,homography=test_homography, flip=False,n_peds_max=1,display_mode=None)
